In [2]:
from pyomo.environ import ConcreteModel, SolverFactory, TransformationFactory
from pyomo.network import Arc
from idaes.core import FlowsheetBlock

In [3]:
import idaes.generic_models.properties.examples.saponification_thermo as thermo_props
import idaes.generic_models.properties.examples.saponification_reactions as reaction_props

In [4]:
from idaes.generic_models.unit_models import CSTR

In [5]:
m = ConcreteModel()

In [6]:
m.fs = FlowsheetBlock(default={"dynamic": False})

In [7]:
m.fs.thermo_params = thermo_props.SaponificationParameterBlock()

In [8]:
m.fs.reaction_params = reaction_props.SaponificationReactionParameterBlock(
                        default={"property_package": m.fs.thermo_params})

In [9]:
m.fs.Tank1 = CSTR(default={"property_package": m.fs.thermo_params,
                           "reaction_package": m.fs.reaction_params,
                           "has_equilibrium_reactions": False,
                           "has_heat_of_reaction": True,
                           "has_heat_transfer": True,
                           "has_pressure_change": False})
m.fs.Tank2 = CSTR(default={"property_package": m.fs.thermo_params,
                           "reaction_package": m.fs.reaction_params,
                           "has_equilibrium_reactions": False,
                           "has_heat_of_reaction": True,
                           "has_heat_transfer": True,
                           "has_pressure_change": False})

In [10]:
m.fs.stream = Arc(source=m.fs.Tank1.outlet,
                  destination=m.fs.Tank2.inlet)

In [11]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [12]:
m.fs.Tank1.inlet.flow_vol[0].fix(1.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "H2O"].fix(55388.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "NaOH"].fix(100.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "EthylAcetate"].fix(100.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "SodiumAcetate"].fix(0.0)
m.fs.Tank1.inlet.conc_mol_comp[0, "Ethanol"].fix(0.0)
m.fs.Tank1.inlet.temperature[0].fix(303.15)
m.fs.Tank1.inlet.pressure[0].fix(101325.0)

In [13]:
m.fs.Tank1.volume.fix(1.0)
m.fs.Tank1.heat_duty.fix(0.0)

m.fs.Tank2.volume.fix(1.0)
m.fs.Tank2.heat_duty.fix(0.0)

In [14]:
m.fs.Tank1.initialize()

2020-07-05 20:59:27 [INFO] idaes.init.fs.Tank1.control_volume.reactions: Initialization Complete.
2020-07-05 20:59:27 [INFO] idaes.init.fs.Tank1.control_volume.properties_out: State Released.
2020-07-05 20:59:27 [INFO] idaes.init.fs.Tank1.control_volume: Initialization Complete
2020-07-05 20:59:27 [INFO] idaes.init.fs.Tank1.control_volume.properties_in: State Released.
2020-07-05 20:59:27 [INFO] idaes.init.fs.Tank1: Initialization Complete: optimal - Optimal Solution Found


In [15]:
m.fs.Tank2.initialize(state_args={
        "flow_vol": 1.0,
        "conc_mol_comp": {"H2O": 55388.0,
                          "NaOH": 100.0,
                          "EthylAcetate": 100.0,
                          "SodiumAcetate": 0.0,
                          "Ethanol": 0.0},
        "temperature": 303.15,
        "pressure": 101325.0})

2020-07-05 21:02:28 [INFO] idaes.init.fs.Tank2.control_volume.reactions: Initialization Complete.
2020-07-05 21:02:28 [INFO] idaes.init.fs.Tank2.control_volume.properties_out: State Released.
2020-07-05 21:02:28 [INFO] idaes.init.fs.Tank2.control_volume: Initialization Complete
2020-07-05 21:02:28 [INFO] idaes.init.fs.Tank2.control_volume.properties_in: State Released.
2020-07-05 21:02:28 [INFO] idaes.init.fs.Tank2: Initialization Complete: optimal - Optimal Solution Found


In [16]:
solver = SolverFactory('ipopt')

In [17]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [18]:
print(results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 40
  Number of variables: 40
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.10309028625488281
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [19]:
print("Tank 1 Outlet")
m.fs.Tank1.outlet.display()
print()
print("Tank 2 Outlet")
m.fs.Tank2.outlet.display()

Tank 1 Outlet
outlet : Size=1
    Key  : Name          : Value
    None : conc_mol_comp : {(0.0, 'Ethanol'): 75.71405668314925, (0.0, 'EthylAcetate'): 24.285943316850748, (0.0, 'H2O'): 55388.0, (0.0, 'NaOH'): 24.285943316850748, (0.0, 'SodiumAcetate'): 75.71405668314925}
         :      flow_vol : {0.0: 1.0}
         :      pressure : {0.0: 101325.0}
         :   temperature : {0.0: 304.0392138960668}

Tank 2 Outlet
outlet : Size=1
    Key  : Name          : Value
    None : conc_mol_comp : {(0.0, 'Ethanol'): 89.63418767673981, (0.0, 'EthylAcetate'): 10.365812323260183, (0.0, 'H2O'): 55388.0, (0.0, 'NaOH'): 10.365812323260183, (0.0, 'SodiumAcetate'): 89.63418767673981}
         :      flow_vol : {0.0: 1.0}
         :      pressure : {0.0: 101325.0}
         :   temperature : {0.0: 304.2026970649369}
